<a href="https://colab.research.google.com/github/abhik-99/MFSGC/blob/master/(Test)Multi_Filter_Supervised_Gene_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(Test)Mulit-filtering Supeprvised Gene Clustering
**Please Execute the Dataset Splitter notebook before coming ot this notebook.**
Upload the *DATASET_test.csv* file obtained from Dataset splitter in this notebook.

In [ ]:
!pip install -U -q PyDrive
!pip install skfeature-chappers

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut

from scipy.sparse import *

In [ ]:
files.upload()

Saving MLL_test.csv to MLL_test.csv


{'MLL_test.csv': b',0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,27

In [ ]:
DATASET="MLL"

data_df=pd.read_csv("%s_test.csv"%(DATASET),index_col=0)
target=data_df.iloc[:,-1]
features=pd.DataFrame(data_df.iloc[:,:-1].values,dtype='float')
m,n=feature.shape
print(m,n)
print(feature.head())
print("Number of classes - ")
classes = np.unique(target)
for x in classes:
  print("Class -",x,"Number of Samples -", len(np.where(target == x)[0]))

feature_norm=pd.DataFrame(MinMaxScaler().fit_transform(feature))

20 2000
      0       1       2     ...        1997       1998       1999
0   8590.0  5470.0  4260.0  ...   75.699997  83.500000  28.700001
1   2510.0  1960.0  1570.0  ...   36.299999   9.820000  21.900000
2   4030.0  3160.0  2870.0  ...   29.799999  44.400002  52.299999
3   5270.0  4740.0  3320.0  ...   67.199997  65.599998  44.000000
4  14200.0  8410.0  6040.0  ...  205.000000  58.200001  44.099998

[5 rows x 2000 columns]
Number of classes - 
Class - 0.0 Number of Samples - 8
Class - 1.0 Number of Samples - 6
Class - 2.0 Number of Samples - 9


In [ ]:
"""
Loading the JSON files from memory
"""
for x in range(3):
  files.upload()
with open("%s_mi_cluster.json"%(DATASET),"r") as fp:
  mi_cluster=json.load(fp)
with open("%s_chi_cluster.json"%(DATASET),"r") as fp:
  chi_cluster=json.load(fp)
with open("%s_relief_cluster.json"%(DATASET),"r") as fp:
  relief_cluster=json.load(fp)

Saving MLL_chi_cluster.json to MLL_chi_cluster.json


Saving MLL_mi_cluster.json to MLL_mi_cluster.json


Saving MLL_relief_cluster.json to MLL_relief_cluster.json


In [ ]:
#construction of ReliefF function

"""
Given a dataset, number of random instances to pick form the dataset and
number of features to consider in each iteration (k), the function returns the weigths of the attributes
of the dataset.
These weigths can then be used as the final results out of the ReliefF algorithm

Paper-

Marko Robnik-ˇSikonja and Igor Kononenko. Theoretical and empirical analysis of relieff
and rrelieff. Machine learning, 53(1-2):23–69, 2003.

"""

def hit_miss_calculator(target,instance,k=10,hit=True,c=None):
    m=len(target)
    upper,lower=instance-1,instance+1
    hits=[]
    hit_flag=False
    #finds k nearest hits
    while(not hit_flag):
      #print(upper,lower)
      if(len(hits)>=k):
        hit_flag = True
        break
      if upper < 0 and lower > m:
        hit_flag = True
        break
      if(upper>=0):
        if((target[upper]==target[instance]) and hit):
          hits.append(upper)
        elif((target[upper]!=target[instance]) and (not hit) and target[upper]==c):
          hits.append(upper)
        upper-=1          
      if(lower<m):
        if((target[lower]==target[instance]) and hit):
          hits.append(lower)
        elif((target[lower]!=target[instance]) and (not hit) and target[lower]==c):
          hits.append(lower)
        lower+=1
    hits.sort()
    return hits


def reliefF(feature,target,k=10,repetitions=10):
  if len(feature.shape)>1:
    m,n=feature.shape
  else:
    m=len(feature)
    n=1
  #print(m,n)
  observations=list(range(m))
  classes=np.unique(target)
  weights=np.zeros(n)
  d=(np.max(feature,axis=0)-np.min(feature,axis=0))*m*k

  for i in range(repetitions):
    instance=np.random.choice(observations,1)[0]
    #print("Iteration",i)
    #print(instance)
    hits=hit_miss_calculator(target,instance,k)
    hit_class_prob=len(np.where(target==target[instance])[0])/m
    #print("\nHit Probability -",hit_class_prob)
    #print("Repetition",i,"Class",target[instance],"Hits -",hits)

    miss={}
    miss_class_prob={}

    for each_class in classes:
      if(each_class != target[instance]):
        miss[each_class]=hit_miss_calculator(target,instance,k,False,each_class)
        class_prob=len(np.where(target==each_class)[0])/m
        #print(each_class,class_prob)
        miss_class_prob[each_class]=hit_class_prob/(1 - (class_prob))

    #print("Repetition",i,"Miss-",miss,"Miss Class Probability -",miss_class_prob)
    
    for hit in hits:
      if len(feature.shape)>1:
        weights-=np.subtract(feature.iloc[instance,:],feature.iloc[hit,:])/d
      else:
        weights-=np.subtract(feature.iloc[instance],feature.iloc[hit])/d
    for each_class in miss:
      for each_miss in miss[each_class]:
        if len(feature.shape)>1:
          weights+=(np.subtract(feature.iloc[instance,:],feature.iloc[each_miss,:])/d)*miss_class_prob[each_class]
        else:
          weights+=(np.subtract(feature.iloc[instance],feature.iloc[each_miss])/d)*miss_class_prob[each_class]
    
    
  return weights.tolist()

In [ ]:
#This function discretizes the given features into 3 categories
def discretize_feature(feature):
  
  mean=np.mean(feature)
  std=np.std(feature)
  discretized=np.copy(feature)
  
  discretized[np.where(feature<(mean+std/2)) ,]=2#within 1/2 std div
  discretized[np.where(feature>(mean-std/2)),]=2#within 1/2 std div
  
  discretized[np.where(feature>(mean+std/2)),]=0#greater than half
  discretized[np.where(feature<(mean-std/2)),]=1#less than half
  
  return discretized

def Xfreq(x):
  xL={}
  for e in x:
    if e not in xL:
      xL[e]=0
    else:
      xL[e]+=1
  for e in xL:
    xL[e]/=len(x)
  return xL

def XYfreq(x,y):
  freq={}
  
  rX=np.unique(x)
  rY=np.unique(y)
      
  for e in rX:
    for f in rY:
      freq[(e,f)]=round(len(np.where(y[np.where(x==e)[0]]==f)[0])/len(x),4)
       
  return freq

def mutual_info(x,y):

  xFreq=Xfreq(x)
  yFreq=Xfreq(y)
  joint=XYfreq(x,y)
  
  Xentropy=0
  for e in xFreq:
    if xFreq[e]!=0:
      Xentropy-=xFreq[e]*np.log2(xFreq[e])
      
  Yentropy=0
  for e in yFreq:
    if yFreq[e]!=0:
      Yentropy-=yFreq[e]*np.log2(yFreq[e])
      
  jentropy=0
  for e in xFreq:
    for f in yFreq:
      if joint[(e,f)]!=0:
        jentropy-=joint[(e,f)]*np.log2(joint[(e,f)])
  
  return (Xentropy+Yentropy-jentropy)

def mutual_info_wrapper(features,target):

  mi=np.array([])
  for x in features:
    discrete=discretize_feature(features[x])
    mi=np.append(mi,mutual_info(discrete,target))
  return np.array(mi)

In [ ]:
def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [ ]:
"""
Takes a Gene Augmentation Sequence and then creates Augnmented Gene Clusters and their Representatives from it.
Must be supplied with the sequence at the beginning of the Notebook. To create a Gene Sequence, run the Notebook
for "Train". There the sequences are created and downloaded.
"""
def get_augmented(features,target, cluster_sequence):
  clusters=pd.DataFrame()
  for each in cluster_sequence:
    base_gene = features[int(each)]
    for each_gene in cluster_sequence[each]:
      operation = each_gene[-1]
      gene_key = int(each_gene[:-1])
      gene = features[gene_key]
      if operation ==  "+":
        base_gene = np.add(base_gene, gene, dtype = 'float64')
      elif operation == '-':
        base_gene = np.subtract(base_gene, gene, dype = 'float64')
    clusters[int(each)] = base_gene
  return cluster_sequence, clusters
#return value is similar to get_cluster in Train Notebook to make sure compatibility with evaluation code from it.

In [ ]:
mi_cluster, gene_repre_1 = get_augmented(features, target, mi_cluster)
relief_cluster, gene_repre_2 = get_augmented(features, target, relief_cluster)
chi_cluster, gene_repre_3 = get_augmented(features, target, chi_cluster)

In [ ]:
gene_repre_1=pd.DataFrame(gene_repre_1)
gene_repre_2=pd.DataFrame(gene_repre_2)
gene_repre_3=pd.DataFrame(gene_repre_3)

In [ ]:
print(gene_repre_1.head())
print(gene_repre_2.head())
print(gene_repre_3.head())

     10456    11324    7815     5784   ...   9021     7279    9017   5819 
0  29940.0  13488.6  26381.0  33699.7  ... -3956.7  10739.2  2215.7  956.7
1  33696.0  14392.0  40213.0  34113.0  ... -3834.0  23333.0  3818.0  651.0
2  21390.0  12448.0  22191.0  34858.0  ... -8154.0   6038.0  2534.0  557.0
3  24810.0   9780.0  36117.0  31049.0  ... -2938.0  13597.0  2950.0  761.0
4  38044.0  18790.0  31070.0  54673.0  ... -7368.0   9052.0  4246.0  290.0

[5 rows x 27 columns]
      3783    4416    1573     8645  ...    305     9365     8007    3234
0 -24620.2   318.3  1774.9  15444.3  ...   809.0 -1824.2   -950.2  -952.6
1 -21167.0  1128.0  2463.0  14427.0  ...  3671.0 -2783.0 -14621.0  -690.0
2 -19907.0   774.0   964.0  14777.0  ...  2850.0 -1518.0  -5007.0 -1829.0
3 -20511.0  1361.0   530.0  14925.0  ...  1735.0 -4845.0 -23693.0  -704.0
4 -25976.0   838.0  2323.0  11639.0  ...  1508.0 -5380.0  -4696.0  -487.0

[5 rows x 27 columns]
     10456    11324    7815     5784   ...   9021     7279  

In [ ]:
def sort_keys(scores,gene_repre,target,flag=True):
  score_dict={}
  x=0
  for i in gene_repre.columns:
    score_dict[i]=scores[x]
    x+=1
  return [k for k, v in sorted(score_dict.items(), key=lambda item: item[1], reverse = True)]

In [ ]:
"""
feature_ranking cannot be used here because it sorts and returns the indices from 0-1
They need to be sorted using a different function
"""
sorted_mi_keys=sort_keys(mutual_info_wrapper(gene_repre_1,target),gene_repre_1,target,True)

sorted_relief_keys=sort_keys(reliefF(gene_repre_2,target,k=5,repetitions=5),gene_repre_2,target,True)

mms=MinMaxScaler()
nfeature=mms.fit_transform(gene_repre_3)
chi_score,p_val=chi2(nfeature,target)
sorted_chi_keys=sort_keys(chi_score,gene_repre_3,target,False)

In [ ]:
print("MI cluster after sorting - ",sorted_mi_keys)
print("Relief cluster after sorting - ",sorted_relief_keys)
print("Chi cluster after sorting - ",sorted_chi_keys)

MI cluster after sorting -  [6415, 7279, 8936, 2027, 6897, 5784, 3398, 10529, 7028, 5369, 6798, 9021, 1695, 5215, 9920, 7871, 7305, 7097, 9764, 10456, 10559, 11324, 4517, 5626, 7815, 9017, 5819]
Relief cluster after sorting -  [8645, 2526, 314, 125, 9564, 4473, 3234, 1573, 5611, 984, 4635, 5091, 8957, 12514, 5339, 950, 8007, 305, 10557, 4416, 9365, 909, 5349, 2376, 9349, 3783, 632]
Chi cluster after sorting -  [10529, 8936, 6415, 3398, 1695, 6798, 9920, 5784, 5819, 7305, 10456, 7028, 7871, 7279, 6897, 5626, 9017, 5369, 7815, 2027, 7097, 9021, 11324, 4517, 9764, 10559, 5215]


In [ ]:
LOOCV=LeaveOneOut()
data_KNN=KNeighborsClassifier(n_neighbors= int(feature.shape[0] ** 0.5))
data_SVM=SVC(kernel='rbf',gamma='scale')
data_NB=GaussianNB()
data_Tree= DecisionTreeClassifier()
rows=feature.shape[0]
classifiers=["NB","KNN","Tree","SVM"]

keys_list=[sorted_mi_keys,sorted_relief_keys,sorted_chi_keys]
cluster_list=[gene_repre_1,gene_repre_2,gene_repre_3]

In [ ]:
acc_matrix = pd.DataFrame()
for i in range(1,6):
  """
  Make a dataframe out of i keys from each gene_representatives from their 
  respective sorted keys. 
  Than use LOOCV to measure accuracy on Train Dataset.
  """
  cluster_df=pd.DataFrame()
  s=0
  for x in range(3):
    tem_df = cluster_list[x][keys_list[x][:i]]
    for x in range(i):
      cluster_df[s] = tem_df.iloc[:,x]
      s+=1
  print(cluster_df.shape)
  acc=0
  individual_acc = np.zeros(4)
  for train_index,test_index in LOOCV.split(cluster_df):
    """
    Data is divided into train-test splits and then polling method is used 
    to find the classification results (ensemble of KNN,SVM,NB,Decision Tree)
    """
    train_data,train_labels=cluster_df.iloc[train_index,:],target[train_index]
    test_data,test_labels=cluster_df.iloc[test_index,:],target[test_index].values.tolist()[0]
    data_KNN.fit(train_data,train_labels)
    data_SVM.fit(train_data,train_labels)
    data_NB.fit(train_data,train_labels)
    data_Tree.fit(train_data,train_labels)

    class_list = [data_NB, data_KNN, data_Tree, data_SVM]
    results=[]
    for x in range(4):
      tem_result = class_list[x].predict(test_data)[0]
      if tem_result == test_labels:
        individual_acc[x]+=1
      results.append(tem_result)
    polling_result=0
    max_freq=0
    for x in results:
      freq=results.count(x)
      if freq>max_freq:
        max_freq=freq
        polling_result=x
    if polling_result == test_labels:
      acc+=1
  individual_acc = np.round(individual_acc/cluster_df.shape[0],4)
  individual_acc = np.append(individual_acc, np.round(acc/cluster_df.shape[0],4))
  print(individual_acc)
  acc_matrix[i] = individual_acc
acc_matrix = acc_matrix.T
acc_matrix.columns = classifiers[:]+['Ensemble']

(23, 3)
[0.8261 0.6087 0.7391 0.6087 0.7391]
(23, 6)
[0.7826 0.6087 0.8696 0.6087 0.7391]
(23, 9)
[0.8261 0.6957 0.8261 0.6522 0.7826]
(23, 12)
[0.9565 0.8261 0.913  0.7826 0.913 ]
(23, 15)
[0.9565 0.8261 0.9565 0.7391 0.9565]


In [ ]:
print(acc_matrix)
acc_matrix.to_csv("%s_Test_Accuracy_Matrix.csv"%(DATASET))

       NB     KNN    Tree     SVM  Ensemble
1  0.8261  0.6087  0.7391  0.6087    0.7391
2  0.7826  0.6087  0.8696  0.6087    0.7391
3  0.8261  0.6957  0.8261  0.6522    0.7826
4  0.9565  0.8261  0.9130  0.7826    0.9130
5  0.9565  0.8261  0.9565  0.7391    0.9565


In [ ]:
files.download("%s_Test_Accuracy_Matrix.csv"%(DATASET))